# Assignment 2
You will continue to work with the hue data files supplied for Assignment 1. We assume that the folder that you work in has the following structure.
<code>
assignment02.ipynb
hue_upload.csv
hue_upload2.csv
</code>

The first four columns represent the `row id`, `user id`, `event id`, and `value`. Any extra columns are irrelevant. For example, the first row of one file reads:

`"1";"10";"lamp change 29 mei 2015 19 08 33 984";"OFF"`

As you can see, the `event id` encompasses both a description of the event (`lamp_change`) and the date/time
(May 29, 7:08:33 pm). The following events are considered informative:

| String               | Description                                               |
-----------------------|------------------------------------------------------------
| `lamp_change`          | Light control via app                                     |
| `nudge_time`           | Automatic light dim time for people in experimental group |
| `bedtime_tonight`      | Intended bedtime (self-reported)                          |
| `risetime`             | Rise time (self-reported)                                 |
| `rise_reason`          | Reason for rising (self-reported)                         |
| `adherence importance` | Adherence (self-reported)                                 |
| `fitness`              | Fitness (self-reported)                                   |

All self-reported values are entered around noon. Records with other events may be ignored.

In [2]:
import datetime as datetime
import pandas as pd
import numpy as np
import pymongo
import re 

pd.options.display.max_rows = 20

## Exercise 1 (70 points)
The first part of this assignment is to write a Python function `read_csv_data` that reads the data into
a Pandas DataFrame. The index should be a (date, user) tuple, where date is stored in datetime.datetime format
(see the document with the "Tips"). The columns of your Pandas DataFrame should be `bedtime`, `intended_bedtime`,
`rise_time`, `rise_reason`, `fitness`, `adherence_importance` and `in_experimental_group`. Note: it is important to stick to this nomenclature. The way to do this is by going through the CSV data line by line, and parsing each line
individually, following these requirements:

<ul>
<li>
`bedtime` should be inferred from the `lamp_change` event. There are multiple reasonable ways to accomplish this. In this assigment, we define the  bedtime as the last OFF state of the `lamp_change` in the interval between 7 pm of the current day and 6 am of the next day. For example, from the row printed above you may
infer that the person did not sleep before 7:08:33 pm. As you go through the lines in the csv file, whenever
you discover new relevant information, you either update an existing record in the dataframe (if a record
for that day and user exists), or you create a new record (see the document with the "Tips").
<br><br>
For example, if you encounter a line where user 10 turns the light on at 9 pm and another line where he
turns it off at 10 pm (still on May 29), you update the record above to change the bedtime to 10 pm. If
someone falls asleep past midnight, the bedtime should be stored in the record corresponding to the day before. Again, dates and times should be stored as datetime.datetime.
<br><br></li>
<li>`intended_bedtime` should be filled in based on the `bedtime_tonight` event. Note that 1030 probably means 10:30 in the evening. Again, dates and times should be stored as datetime.datetime.
<br><br></li>
<li>
`rise_time`. The value for the column in your solution should be obtained from the `risetime` event in the CSV file.
<br><br></li>
<li>
`rise_reason`, `fitness` and `adherence_importance` values should be copied from the CSV file. Note that if
multiple distinct values are entered, the last should be assumed to be correct.
<br><br></li>
<li>
`in_experimental_group` should be boolean (True/False). The default value is False, but should be changed
to True if a `nudge_time` event is encountered. If a user is in the experimental group on one day, he is on
all days.
</li>
</ul>    

In [3]:
def read_csv_data(filenames):
# YOUR CODE HERE
    import locale
    locale.setlocale(locale.LC_TIME, 'nl_NL.utf8')

    def insert(df, idx, exp):
        if idx not in df.index:
            df = df.append(pd.Series({'bedtime' : pd.NaT, 'intended_bedtime' : pd.NaT, \
                                'rise_time' : pd.NaT, 'rise_reason' : "", \
                                'fitness' : float('nan'), 'adherence_importance' : float('nan'), \
                                'in_experimental_group' : idx[1] in exp}, name=idx))
        return df

    def update(df, idx, row):
        event, lamp_time = row['event_id'], row['time_event_id']

        if 'lamp_change' in event and not pd.isnull(lamp_time):
            time_from = datetime.datetime.strptime("19", "%H")
            time_to = datetime.datetime.strptime("6", "%H")
            if lamp_time >= time_from or lamp_time <= time_to:
                old_time = df.at[idx, 'bedtime']
                if (not pd.isnull(old_time) and lamp_time > old_time) or pd.isnull(old_time):
                        df.at[idx, 'bedtime'] = lamp_time 
        elif 'fitness' in event:
            df.at[idx, 'fitness'] = row['value']
        elif 'adherence' in event:
            df.at[idx, 'adherence_importance'] = row['value']
        elif 'rise_reason' in event:
            df.at[idx, 'rise_reason'] = row['value']
        elif 'bedtime_tonight' in event:
            df.at[idx, 'intended_bedtime'] = row['value_time_event']
        elif 'risetime' in event:
            df.at[idx, 'rise_time'] = row['value_time_event']

        return df
    
    
    # parser of bedtime and risetime 
    def time_parser(row):
        if 'bedtime_tonight' in row['event_id'] or 'risetime' in row['event_id']:
            item = str(row['value'])
            if item != 'nan':
                if len(item) >= 5:
                    filled = item.zfill(6)
                    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
                elif len(item) == 4:
                    new = (':'.join(item[i:i+2] for i in range(0, len(item), 2)))
                    new += ':00'
                    if (new == '24:00:00'):
                        new = '00:00:00'
                elif len(item) == 3:
                    filled = item.zfill(4)
                    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
                    new += ':00'
                elif len(item) == 2:
                    filled = item.zfill(4)
                    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
                    new += ':00'
                elif len(item) == 1:
                    filled = item.zfill(4)
                    new = (':'.join(filled[i:i+2] for i in range(0, len(filled), 2)))
                    new += ':00'
                return datetime.datetime.strptime(new, "%H:%M:%S")
 


    # Fileinput
    dataframes = []
    colnames = ['row_id', 'user_id', 'event_id', 'value']
    for filename in filenames:
        dataframes.append(pd.read_csv(filename, sep = ';', names = colnames, header = None, encoding ='utf_8'))
    df = pd.concat(dataframes)
        
    # Filter informative events
    key_words = ['nudge_time', 'lamp_change', 'bedtime_tonight', 'risetime', 'rise_reason',
                'adherence_importance', 'fitness']
    key_regex = '.*(' + '|'.join(key_words) + ')'
    df = df.loc[df.event_id.str.match(key_regex)]
    
    # Pulling out 'Date' from the Event_ID column
    df["date_event_id"] = df["event_id"].str.findall(r"_\d+_\w+_\d{4}").astype(str).str.replace('\[|\]|\'', '')
    df["date_event_id"] = df["date_event_id"].str.replace('_'," ").str.strip()
    df["date_event_id"] = df.apply(lambda x : datetime.datetime.strptime(x["date_event_id"], '%d %B %Y'), axis=1)
    
    # Lamp change time
    df["time_event_id"] = df["event_id"].str.findall(r"_\d{2}_\d{2}_\d{2}").astype(str).str.replace('\[|\]|\'', '')
    df["time_event_id"] = df["time_event_id"].str.replace('_'," ").str.strip()
    df["time_event_id"] = df["time_event_id"].str.replace(" ",":")
    df.replace(r'^\s*$', np.nan, regex=True, inplace = True)
    df['time_event_id'].fillna(value = "00:00:00", inplace = True) 
    df["time_event_id"] = df.apply(lambda x : datetime.datetime.strptime(x["time_event_id"], '%H:%M:%S'), axis=1)
    
    # Rise and bed time parse
    df['value_time_event'] = df.apply(time_parser, axis=1)
    #Create new dataframe
    df_ = pd.DataFrame(data=[])    
    
    experimental_group = set()
    
    for _, row in df.iterrows():
        user, date, event = row['user_id'], row['date_event_id'], row['event_id']
        idx = (date, user)
        if 'nudge_time' in event:
            experimental_group.add(user)
    
    for _, row in df.iterrows():
        user, date = row['user_id'], row['date_event_id']
        idx = (date, user)
        df_ = insert(df_, idx, experimental_group)
        df_ = update(df_, idx, row)
        
    df_['in_experimental_group'] = df_['in_experimental_group'].astype(bool)
    
    return df_

# YOUR CODE ENDS HERE

In [4]:
df = read_csv_data(['hue_upload.csv', 'hue_upload2.csv'])
df

,adherence_importance,bedtime,fitness,in_experimental_group,intended_bedtime,rise_reason,rise_time
"(2015-05-29 00:00:00, 10)",NaN,1900-01-01 19:08:33,NaN,True,NaT,,NaT
"(2015-05-31 00:00:00, 10)",100.0,1900-01-01 23:56:41,52.0,True,1900-01-01 23:00:00,ja,1900-01-01 09:00:00
"(2015-05-28 00:00:00, 37)",NaN,1900-01-01 21:45:42,NaN,False,NaT,,NaT
"(2015-05-31 00:00:00, 37)",11.0,NaT,34.0,False,1900-01-01 12:00:00,ja,1900-01-01 08:30:00
"(2015-05-31 00:00:00, 12)",78.0,1900-01-01 23:21:33,23.0,True,1900-01-01 23:30:00,ja,1900-01-01 12:00:00
...,...,...,...,...,...,...,...
"(2015-09-19 00:00:00, 61)",12.0,1900-01-01 00:53:57,19.0,False,1900-01-01 23:00:00,nee,1900-01-01 11:00:00
"(2015-09-19 00:00:00, 55)",51.0,1900-01-01 21:03:38,30.0,False,1900-01-01 00:30:00,nee,1900-01-01 11:00:00
"(2015-09-19 00:00:00, 58)",47.0,NaT,33.0,False,1900-01-01 23:00:00,nee,1900-01-01 07:00:00
"(2015-09-19 00:00:00, 63)",16.0,NaT,54.0,False,1900-01-01 00:00:00,nee,1900-01-01 09:45:00


In [5]:
# check the index
print(df.index)


Index([(2015-05-29 00:00:00, 10), (2015-05-31 00:00:00, 10),
       (2015-05-28 00:00:00, 37), (2015-05-31 00:00:00, 37),
       (2015-05-31 00:00:00, 12), (2015-06-01 00:00:00, 12),
       (2015-05-29 00:00:00, 12), (2015-06-06 00:00:00, 12),
       (2015-06-02 00:00:00, 12), (2015-06-04 00:00:00, 12),
       ...
       (2015-09-17 00:00:00, 63), (2015-09-18 00:00:00, 61),
       (2015-09-18 00:00:00, 55), (2015-09-18 00:00:00, 58),
       (2015-09-18 00:00:00, 63), (2015-09-19 00:00:00, 61),
       (2015-09-19 00:00:00, 55), (2015-09-19 00:00:00, 58),
       (2015-09-19 00:00:00, 63), (2015-09-20 00:00:00, 55)],
      dtype='object', length=468)


In [6]:
# check the bedtime column
display(df[['bedtime']])


,bedtime
"(2015-05-29 00:00:00, 10)",1900-01-01 19:08:33
"(2015-05-31 00:00:00, 10)",1900-01-01 23:56:41
"(2015-05-28 00:00:00, 37)",1900-01-01 21:45:42
"(2015-05-31 00:00:00, 37)",NaT
"(2015-05-31 00:00:00, 12)",1900-01-01 23:21:33
...,...
"(2015-09-19 00:00:00, 61)",1900-01-01 00:53:57
"(2015-09-19 00:00:00, 55)",1900-01-01 21:03:38
"(2015-09-19 00:00:00, 58)",NaT
"(2015-09-19 00:00:00, 63)",NaT


In [7]:
# check the intended_bedtime column
display(df[['intended_bedtime']])


,intended_bedtime
"(2015-05-29 00:00:00, 10)",NaT
"(2015-05-31 00:00:00, 10)",1900-01-01 23:00:00
"(2015-05-28 00:00:00, 37)",NaT
"(2015-05-31 00:00:00, 37)",1900-01-01 12:00:00
"(2015-05-31 00:00:00, 12)",1900-01-01 23:30:00
...,...
"(2015-09-19 00:00:00, 61)",1900-01-01 23:00:00
"(2015-09-19 00:00:00, 55)",1900-01-01 00:30:00
"(2015-09-19 00:00:00, 58)",1900-01-01 23:00:00
"(2015-09-19 00:00:00, 63)",1900-01-01 00:00:00


In [8]:
# check the rise_time
display(df[['rise_time']])


,rise_time
"(2015-05-29 00:00:00, 10)",NaT
"(2015-05-31 00:00:00, 10)",1900-01-01 09:00:00
"(2015-05-28 00:00:00, 37)",NaT
"(2015-05-31 00:00:00, 37)",1900-01-01 08:30:00
"(2015-05-31 00:00:00, 12)",1900-01-01 12:00:00
...,...
"(2015-09-19 00:00:00, 61)",1900-01-01 11:00:00
"(2015-09-19 00:00:00, 55)",1900-01-01 11:00:00
"(2015-09-19 00:00:00, 58)",1900-01-01 07:00:00
"(2015-09-19 00:00:00, 63)",1900-01-01 09:45:00


In [12]:
# check the rise_reason, fitness, adherence_importance column
display(df[['rise_reason', 'fitness', 'adherence_importance']])


,rise_reason,fitness,adherence_importance
"(2015-05-29 00:00:00, 10)",,NaN,NaN
"(2015-05-31 00:00:00, 10)",ja,52.0,100.0
"(2015-05-28 00:00:00, 37)",,NaN,NaN
"(2015-05-31 00:00:00, 37)",ja,34.0,11.0
"(2015-05-31 00:00:00, 12)",ja,23.0,78.0
...,...,...,...
"(2015-09-19 00:00:00, 61)",nee,19.0,12.0
"(2015-09-19 00:00:00, 55)",nee,30.0,51.0
"(2015-09-19 00:00:00, 58)",nee,33.0,47.0
"(2015-09-19 00:00:00, 63)",nee,54.0,16.0


In [13]:
# check the in_experimental_group column
display(df[['in_experimental_group']])


,in_experimental_group
"(2015-05-29 00:00:00, 10)",True
"(2015-05-31 00:00:00, 10)",True
"(2015-05-28 00:00:00, 37)",False
"(2015-05-31 00:00:00, 37)",False
"(2015-05-31 00:00:00, 12)",True
...,...
"(2015-09-19 00:00:00, 61)",False
"(2015-09-19 00:00:00, 55)",False
"(2015-09-19 00:00:00, 58)",False
"(2015-09-19 00:00:00, 63)",False


## Exercise 2 (10 + 20 points)
The second part of this assignment is to store the contents of the DataFrame into MongoDB, and to write a function that retrieves data from MongoDB and outputs it in a user-friendly format.

<ol>
<li>
The data should be stored in the collection "sleepdata" in the database "BigData". Make sure to use
the same column names as specified for the DataFrame, and to define the correct primary key. See
the document with the "Tips" for some comments about the primary key. Add the extra columns "date",
"user", "sleep duration" to facilitate sorting the data if necessary. Here, "sleep duration" is the difference between the risetime and the bedtime.
<br><br></li>
<li>The following is an example of how the output must be presented.
    
| date | user | bedtime | intended | risetime | reason | fitness | adh | in_exp | sleep_duration |
-------|------|---------|----------|----------|--------|---------|-----|--------|----------------|    
| 11-06-2015 | 2  | 00:51:28 | 22:30:00 | 07:00:00 | ja  | -    | 47.0 | no  | 22351 |
| 11-06-2015 | 20 | 00:28:10 | 23:00:00 | 07:10:00 | nee | 55.0 | 88.0 | yes | 33510 |
| 11-06-2015 | 34 | 19:54:10 | -        | -        | -   | -    | -    | yes | -     |

Here sleep duration is in number of seconds. Note that, in order to determine the sleep duration of day X, it
is necessary to know the risetime of day X, but the bedtime of day X - 1.
<br><br></li>
</ol>

In [14]:
def to_mongodb(df_):
# YOUR CODE HERE
    from datetime import timedelta
    from copy import copy
    df = copy(df_)
    
    # connect to MongoDB database
    client = pymongo.MongoClient("localhost", 27017)
    db = client.BigData
    sleepdata = db.sleepdata
    sleepdata.delete_many({})
    
    # set primary key
    df['_id'] = pd.Series(dict(zip(df.index, list(map(lambda x : {"date": x[0], "name": x[1]}, df.index)))))
    
    # add column date and user
    
    user = pd.Series(dict(zip(df.index, list(map(lambda x : x[1], df.index)))))
    date = pd.Series(dict(zip(df.index, list(map(lambda x : x[0], df.index)))))
    df.insert(loc=0, column='date', value=date)
    df.insert(loc=1, column='user', value=user)
    
    
    # add sleep_duration column
    bedtime = df['bedtime'].transpose().to_dict()
    risetime = df['rise_time'].transpose().to_dict()
    
    def sleep_time(index):
        rise = risetime.get(index)
        bet = bedtime.get((index[0] - timedelta(days=1), index[1]))
        if not pd.isnull(rise) and not pd.isnull(bet):
            return int((rise - bet + timedelta(days=1)).total_seconds())

    
    df['sleep_duration'] = pd.Series(dict(zip(df.index, list(map(sleep_time, df.index)))))
    df.fillna("-",inplace=True)
    sleepdata.insert_many(df.to_dict('records'))
    
# YOUR CODE ENDS HERE

In [15]:
to_mongodb(df)


In [16]:
def read_mongodb(filter,sort):
# YOUR CODE HERE
    
   
    # connect to MongoDB database
    connection = pymongo.MongoClient("localhost", 27017)
    db = connection.BigData
    sleepdata = db.sleepdata
    data = sleepdata.find(filter).sort(sort, pymongo.ASCENDING)   
  
    col = list(sleepdata.find_one({}).keys())
    col.remove('_id')
    
    only_date_col = ['date']
    only_time_col = ['intended_bedtime', 'rise_time', 'bedtime']
    boolean_col = ['in_experimental_group']
    
    def name(col):
        if col == "in_experimental_group":
               return "in_exp"
        if col == "adherence_importance":
            return "adh"
        if col == "rise_reason":
            return "reason"
        if col == "intended_bedtime":
            return "inteded"
        return col
        
    def convert_time(row):
        for i in only_date_col:
            if row[i] != "-":
                row[i] = row[i].date()
        for i in only_time_col:
            if row[i] != "-":
                row[i] = row[i].strftime("%H:%M:%S")
        return row
    
    def convert_bool(row):
        r = {True: 'Yes', False: 'No'}
        for i in boolean_col:
            row[i] = r[row[i]]
        return row
    
    
            
    def col_size(col):
        if col in only_date_col:
            return max(10, len(name(col)))
        if col in only_time_col:
            return max(8, len(name(col)))
        if col == "adherence_importance":
            return 5
        else:
            return len(name(col))
    
    format = "   ".join(list(map(lambda x : "%{}s".format(col_size(x)), col)))
    print(format % tuple(map(name, col)))
    for row in data:
        row = (convert_bool(convert_time(row)))
        
        print(format % tuple(map(lambda x : str(row[x])[0:col_size(x)] if row[x] and row[x] != "" else "-", col)))
    
    return list(data)
# YOUR CODE ENDS HERE

In [17]:
query = read_mongodb({'sleep_duration': {'$gt': 40000}}, '_id')
print(query)


      date   user     adh    bedtime   fitness   in_exp    inteded   reason   rise_time   sleep_duration
2015-05-31      9    50.0   23:26:09      29.0       No   23:00:00       ja    08:30:00          40359.0
2015-05-31     24    75.0          -       6.0       No   22:00:00       ja    10:15:00          40020.0
2015-06-01      1    80.0   03:09:10      81.0       No   00:00:00       ja    11:00:00          54626.0
2015-06-01     31    79.0   23:23:35      78.0       No   23:45:00       ja    08:15:00          41591.0
2015-06-02      1    51.0   01:15:06      68.0       No   01:30:00       ja    08:45:00         106550.0
2015-06-02     18    67.0   20:52:22      63.0      Yes   23:00:00       ja    08:00:00          40324.0
2015-06-02     19    73.0   23:58:25      19.0       No   23:30:00       ja    08:30:00          40818.0
2015-06-02     26    29.0   21:39:26      54.0      Yes   22:45:00      nee    09:30:00          47817.0
2015-06-02     30       -   23:53:27      70.0       No